In [14]:
import sys
libpath = "/home/pablo/tesis/tsm-pipeline/code"
if libpath not in sys.path:
    sys.path.append(libpath)
    
%load_ext autoreload
%autoreload 2

import docker
docker_client = docker.from_env()

# Configura todo lo necesario en el ambiente
from scripts.docker import run_tsm, ExperimentSettings, read_dbs_dataset
from scripts.evaluate import evaluate, EvaluationSettings
from os.path import join as path_join
import logging

logging.basicConfigcConfig(level=logging.INFO)

defaults = {
    "bash_config_path": "/home/pablo/tesis/tsm-pipeline/code/scripts/config.sh",
}
defaults_evalute = {
    "cache_root":"/home/pablo/dbcache",    
}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
nosql_dbs = read_dbs_dataset("/home/pablo/tesis/tsm-pipeline/experiments/datasets/nosql_1.txt")

In [9]:
from sklearn.model_selection import train_test_split
from numpy.random import RandomState

# Using a fixed random state to have reproducible results
rand = RandomState(seed=5)

sample = nosql_dbs[:100]
train, test = train_test_split(sample, train_size=.5, shuffle=True, random_state=rand)
train[:10]

['bookbrainz/bookbrainz-site/d4381bb',
 'JiniHendrix/expensereportapp/db393a6',
 'andrehNSFW/nameless-warrior-api/e7f787d',
 'Redsmin/proxy/0b20436',
 'architMahto/udemy-nodejs-angular2/b1ed580',
 'appbaseio/dejavu/a735ffb',
 'LiJunpeng/URLShortenerService/5c0aa71',
 'andchir/shopkeeper4/ac64a5b',
 'OmarElGabry/chat.io/082952e',
 'DBULL7/uhoops/ffd7a04']

Primera corrida de bejamin button, con un dataset formado a partir de repos que venian con commits, y filtrando que tengan por lo menos 5k loc de javascript. Se toma un corte de 50/50 entre entrenamiento y test.

In [10]:
results_dir = "/home/pablo/results/ds_nosql_1"

In [5]:
results_dir = '/home/pablo/results/seldonstar_nosql_1'

In [11]:
# training
train_settings = ExperimentSettings(
    name="ds_nosql_1",
    query_type="nosql",
    project_list=train,
    results_dir=results_dir,
    **defaults
)

run_tsm(docker_client, train_settings, block=True)

running at container f6e21cff2d3ec89b824fe4805adbdb3e6394f2b1795a94d8bbb39cfc8c00da3a. Use `docker logs f6e21cff2d3ec89b824fe4805adbdb3e6394f2b1795a94d8bbb39cfc8c00da3a --tail 10 --follow` to follow progress
g: /home/pablo/results/ds_nosql_1/*/NosqlInjectionWorse-*/reprScores.txt


In [12]:
!cat /home/pablo/results/ds_nosql_1/averaged-results.csv|wc -l

51


In [13]:
test[0:10]

['PaulSec/VNWA/d3704e6',
 'FreetailHackers/saffron/f563dd2',
 'JosipReh/Feeddit/bbcb615',
 'EQuimper/nodejs-api-boilerplate/0dcea20',
 'Xerios/mobx-isomorphic-starter/8d08656',
 'DanialK/PassportJS-Authentication/af6f691',
 'BryanKo/Fam/6ee0fe0',
 'DanWahlin/Angular-RESTfulService/bbd42e4',
 'Crizstian/cinema-microservice/b6c973d',
 'Gingernaut/Robot-Lawyer/56b86ac']

In [15]:
# evaluation
external_predicate_file = path_join(results_dir, 'averaged-results.csv')
worse_settings = EvaluationSettings(
    search_path="/home/pablo/tesis/tsm-pipeline/lib-worse/codeql/javascript/ql:/home/pablo/.codeql/packages/codeql/javascript-upgrades/0.0.3",
    cli_version="2.13.1",
    db_cli_version="2.5.2",
    query_file="/home/pablo/tesis/tsm-pipeline/tsm-atm-pipeline/src/tsm/evaluation/NosqlInjectionWorseTSM.ql",
    external_predicate_file=external_predicate_file,
    **defaults_evalute,
)

print("Evaluating worse with %d dbs" % (len(test)))
evaluate(
    settings=worse_settings,
    output_dir=path_join(results_dir, 'worse'),
    dbs=test,
)

INFO:scripts.evaluate:creating /home/pablo/results/ds_nosql_1/worse dir. Omitting error if already existing


Evaluating worse with 50 dbs


INFO:scripts.evaluate:Starting processing with 4 processes
INFO:scripts.evaluate:Evaluating Xerios - mobx-isomorphic-starter
INFO:scripts.evaluate:Evaluating PaulSec - VNWA
INFO:scripts.evaluate:Evaluating Crizstian - cinema-microservice
INFO:scripts.evaluate:Evaluating StudioDeTelevision - admiraljs
INFO:scripts.evaluate:Evaluating FreetailHackers - saffron
INFO:scripts.evaluate:Evaluating Gingernaut - Robot-Lawyer
INFO:scripts.evaluate:Evaluating DanialK - PassportJS-Authentication
INFO:scripts.evaluate:Evaluating IBM - innovate-digital-bank
INFO:scripts.evaluate:Evaluating bitpay - bitcore
INFO:scripts.evaluate:Evaluating BryanKo - Fam
INFO:scripts.evaluate:Evaluating JosipReh - Feeddit
INFO:scripts.evaluate:Evaluating OmgImAlexis - astro
INFO:scripts.evaluate:Evaluating DanWahlin - Angular-RESTfulService
INFO:scripts.evaluate:Evaluating EQuimper - nodejs-api-boilerplate
INFO:scripts.evaluate:Evaluating FrontendMasters - intro-mongo-db
INFO:scripts.evaluate:Evaluating IHTSDO - sct-s

In [16]:
v0_settings = EvaluationSettings(
    search_path="/home/pablo/codeqlv0/javascript/ql/lib",
    cli_version="2.13.1",
    db_cli_version="2.5.2",
    query_file="/home/pablo/tesis/tsm-pipeline/tsm-evaluation/tsm-evaluation/src/NoSqlEvaluation.ql",
    **defaults_evalute,
)

print("Evaluating v0 with %d dbs" % (len(test)))
evaluate(
    settings=v0_settings,
    output_dir=path_join(results_dir, 'v0'),
    dbs=test,
)

INFO:scripts.evaluate:creating /home/pablo/results/ds_nosql_1/v0 dir. Omitting error if already existing


Evaluating v0 with 50 dbs


INFO:scripts.evaluate:Starting processing with 4 processes
INFO:scripts.evaluate:Evaluating PaulSec - VNWA
INFO:scripts.evaluate:Evaluating Crizstian - cinema-microservice
INFO:scripts.evaluate:Evaluating StudioDeTelevision - admiraljs
INFO:scripts.evaluate:Evaluating Xerios - mobx-isomorphic-starter
INFO:scripts.evaluate:Evaluating Gingernaut - Robot-Lawyer
INFO:scripts.evaluate:Evaluating DanialK - PassportJS-Authentication
INFO:scripts.evaluate:Evaluating FreetailHackers - saffron
INFO:scripts.evaluate:Evaluating IBM - innovate-digital-bank
INFO:scripts.evaluate:Evaluating BryanKo - Fam
INFO:scripts.evaluate:Evaluating bitpay - bitcore
INFO:scripts.evaluate:Evaluating JosipReh - Feeddit
INFO:scripts.evaluate:Evaluating OmgImAlexis - astro
INFO:scripts.evaluate:Evaluating DanWahlin - Angular-RESTfulService
INFO:scripts.evaluate:Evaluating EQuimper - nodejs-api-boilerplate
INFO:scripts.evaluate:Evaluating FrontendMasters - intro-mongo-db
INFO:scripts.evaluate:Evaluating IHTSDO - sct-s

In [19]:
from scripts.calculate_scores import calculate_scores_df

calculate_scores_df(
    results_folder=results_dir,
    cleanup_base_dir="/tmp",
)

,precision,recall,accuracy,alerts to recover,alerts recovered,suprious alerts
0,0.0,0.0,0.0,21,0,304
